In [3]:
import datetime as dt
import pandas as pd
import matplotlib as plt


In [10]:
pip install Lifetimes

  Using cached Lifetimes-0.11.3-py3-none-any.whl (584 kB)
  Using cached autograd-1.5-py3-none-any.whl (48 kB)
Note: you may need to restart the kernel to use updated packages.


In [11]:
from lifetimes import BetaGeoFitter
from lifetimes import GammaGammaFitter
from lifetimes.plotting import plot_period_transactions

In [71]:
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)
pd.set_option('display.float_format', lambda x: '%.2f' % x)
from sklearn.preprocessing import MinMaxScaler

master_id: Eşsiz müşteri numarası
order_channel: Alışveriş yapılan platforma ait hangi kanalın kullanıldığı (Android, ios, Desktop, Mobile)
last_order_channel: En son alışverişin yapıldığı kanal
first_order_date: Müşterinin yaptığı ilk alışveriş tarihi
last_order_date: Müşterinin yaptığı son alışveriş tarihi
last_order_date_online: Müşterinin online platformda yaptığı son alışveriş tarihi
last_order_date_offline: Müşterinin offline platformda yaptığı son alışveriş tarihi
order_num_total_ever_online: Müşterinin online platformda yaptığı toplam alışveriş sayısı
order_num_total_ever_offline: Müşterinin offline'da yaptığı toplam alışveriş sayısı
customer_value_total_ever_offline: Müşterinin offline alışverişlerinde ödediği toplam ücret
customer_value_total_ever_online: Müşterinin online alışverişlerinde ödediği toplam ücret
interested_in_categories_12: Müşterinin son 12 ayda alışveriş yaptığı kategorilerin listei

Adım 1: flo_data_20K.csv verisini okuyunuz.Dataframe’in kopyasını oluşturunuz

In [17]:
df_ = pd.read_csv("flo_data_20k.csv")

In [18]:
df = df_.copy()

Adım 2: Veri setinde
a. İlk 10 gözlem,
b. Değişken isimleri,
c. Betimsel istatistik,
d. Boş değer,
e. Değişken tipleri, incelemesi yapınız.


In [22]:
df.head(10) #İlk 10 gözlem
df.columns #Değişken isimleri
df.describe() #Betimsel istatistik
df.isnull().sum() #Boş değer
df.info() #Değişken tipleri







<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19945 entries, 0 to 19944
Data columns (total 12 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   master_id                          19945 non-null  object 
 1   order_channel                      19945 non-null  object 
 2   last_order_channel                 19945 non-null  object 
 3   first_order_date                   19945 non-null  object 
 4   last_order_date                    19945 non-null  object 
 5   last_order_date_online             19945 non-null  object 
 6   last_order_date_offline            19945 non-null  object 
 7   order_num_total_ever_online        19945 non-null  float64
 8   order_num_total_ever_offline       19945 non-null  float64
 9   customer_value_total_ever_offline  19945 non-null  float64
 10  customer_value_total_ever_online   19945 non-null  float64
 11  interested_in_categories_12        19945 non-null  obj

Adım 3: Omnichannel müşterilerin hem online'dan hemde offline platformlardan alışveriş yaptığını ifade etmektedir. Her bir müşterinin toplam alışveriş sayısı ve harcaması için yeni değişkenler oluşturunuz.

In [38]:
df["num_total"]=df["order_num_total_ever_online"]+df["order_num_total_ever_offline"] # toplam alışveriş sayısı
df["value_total"]=df["customer_value_total_ever_offline"]+df["customer_value_total_ever_online"] #toplam harcama
df.head()


,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12,num_total,value_total
0,cc294636-19f0-11eb-8d74-000d3a38a36f,Android App,Offline,2020-10-30,2021-02-26,2021-02-21,2021-02-26,4.0000,1.0000,139.9900,799.3800,[KADIN],5.0000,939.3700
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,Android App,Mobile,2017-02-08,2021-02-16,2021-02-16,2020-01-10,19.0000,2.0000,159.9700,1853.5800,"[ERKEK, COCUK, KADIN, AKTIFSPOR]",21.0000,2013.5500
2,69b69676-1a40-11ea-941b-000d3a38a36f,Android App,Android App,2019-11-27,2020-11-27,2020-11-27,2019-12-01,3.0000,2.0000,189.9700,395.3500,"[ERKEK, KADIN]",5.0000,585.3200
3,1854e56c-491f-11eb-806e-000d3a38a36f,Android App,Android App,2021-01-06,2021-01-17,2021-01-17,2021-01-06,1.0000,1.0000,39.9900,81.9800,"[AKTIFCOCUK, COCUK]",2.0000,121.9700
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,Desktop,Desktop,2019-08-03,2021-03-07,2021-03-07,2019-08-03,1.0000,1.0000,49.9900,159.9900,[AKTIFSPOR],2.0000,209.9800


Adım 4: Değişken tiplerini inceleyiniz. Tarih ifade eden değişkenlerin tipini date'e çeviriniz.

In [51]:
df.info()

#Aşağıda değişklik yapmak istediğimiz kolonları seçtik ve pandasdan datatime'a  çevirdik.

df[["first_order_date", "last_order_date", "last_order_date_online","last_order_date_offline"]] =df[["first_order_date",
                                                                                                     "last_order_date",
                                                                                                     "last_order_date_online", 
                                                                                                     "last_order_date_offline"]].apply(pd.to_datetime)

#2. yol aşağıdaki gibi yaparsak içinde "date" geçen colonları "datetime" a çevir yapabiliriz.

#date_columns = df.columns[df.columns.str.contains("date")]
#df[date_columns] = df[date_columns].apply(pd.to_datetime)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19945 entries, 0 to 19944
Data columns (total 14 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   master_id                          19945 non-null  object        
 1   order_channel                      19945 non-null  object        
 2   last_order_channel                 19945 non-null  object        
 3   first_order_date                   19945 non-null  datetime64[ns]
 4   last_order_date                    19945 non-null  datetime64[ns]
 5   last_order_date_online             19945 non-null  datetime64[ns]
 6   last_order_date_offline            19945 non-null  datetime64[ns]
 7   order_num_total_ever_online        19945 non-null  float64       
 8   order_num_total_ever_offline       19945 non-null  float64       
 9   customer_value_total_ever_offline  19945 non-null  float64       
 10  customer_value_total_ever_online  

Adım 5: Alışveriş kanallarındaki müşteri sayısının, toplam alınan ürün sayısının ve toplam harcamaların dağılımına bakınız.

In [52]:
df.groupby("order_channel").agg({"master_id":"count",
                                 "num_total":"sum",
                                 "value_total":"sum"})
#Alışveriş kanallarına göre dağılımlarına bakıyoruz.

,master_id,num_total,value_total
order_channel,,,
Android App,9495,52269.0000,7819062.7600
Desktop,2735,10920.0000,1610321.4600
Ios App,2833,15351.0000,2525999.9300
Mobile,4882,21679.0000,3028183.1600


Adım 6: En fazla kazancı getiren ilk 10 müşteriyi sıralayınız.

In [56]:
df.sort_values("value_total", ascending=False)[:10] #sort_values kullanarak belirtilen kolona göre sıralama yaptık

,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12,num_total,value_total
11150,5d1c466a-9cfd-11e9-9897-000d3a38a36f,Android App,Desktop,2013-10-11,2021-04-30,2021-04-30,2020-12-24,200.0000,2.0000,684.9700,45220.1300,"[AKTIFCOCUK, ERKEK, COCUK, KADIN, AKTIFSPOR]",202.0000,45905.1000
4315,d5ef8058-a5c6-11e9-a2fc-000d3a38a36f,Android App,Android App,2018-08-06,2021-02-23,2021-02-23,2020-07-06,67.0000,1.0000,130.4900,36687.8000,"[AKTIFCOCUK, ERKEK, KADIN, AKTIFSPOR]",68.0000,36818.2900
7613,73fd19aa-9e37-11e9-9897-000d3a38a36f,Ios App,Offline,2014-01-14,2021-05-18,2021-01-30,2021-05-18,81.0000,1.0000,1263.7600,32654.3400,"[ERKEK, COCUK, KADIN, AKTIFSPOR]",82.0000,33918.1000
13880,7137a5c0-7aad-11ea-8f20-000d3a38a36f,Ios App,Offline,2021-03-01,2021-04-13,2021-03-18,2021-04-13,10.0000,1.0000,538.9400,30688.4700,"[ERKEK, KADIN, AKTIFSPOR]",11.0000,31227.4100
9055,47a642fe-975b-11eb-8c2a-000d3a38a36f,Android App,Offline,2021-04-07,2021-04-27,2021-04-07,2021-04-27,1.0000,3.0000,18119.1400,2587.2000,[AKTIFSPOR],4.0000,20706.3400
7330,a4d534a2-5b1b-11eb-8dbd-000d3a38a36f,Desktop,Desktop,2020-02-16,2021-04-30,2021-04-30,2020-12-18,66.0000,4.0000,843.6800,17599.8900,"[ERKEK, KADIN, AKTIFSPOR]",70.0000,18443.5700
8068,d696c654-2633-11ea-8e1c-000d3a38a36f,Ios App,Ios App,2017-05-10,2021-04-13,2021-04-13,2019-08-15,69.0000,1.0000,82.4800,16836.0900,"[AKTIFCOCUK, ERKEK, COCUK, KADIN, AKTIFSPOR]",70.0000,16918.5700
163,fef57ffa-aae6-11e9-a2fc-000d3a38a36f,Mobile,Desktop,2016-11-08,2021-05-12,2021-05-12,2020-07-09,36.0000,1.0000,180.7300,12545.3700,"[ERKEK, AKTIFSPOR]",37.0000,12726.1000
7223,cba59206-9dd1-11e9-9897-000d3a38a36f,Android App,Android App,2013-02-21,2021-05-09,2021-05-09,2020-01-25,130.0000,1.0000,49.9900,12232.2500,"[AKTIFCOCUK, ERKEK, KADIN, AKTIFSPOR]",131.0000,12282.2400
18767,fc0ce7a4-9d87-11e9-9897-000d3a38a36f,Desktop,Desktop,2018-11-24,2020-11-11,2020-11-11,2019-12-06,18.0000,2.0000,64.9700,12038.1800,"[ERKEK, KADIN]",20.0000,12103.1500


Adım 7: En fazla siparişi veren ilk 10 müşteriyi sıralayınız.

In [57]:
df.sort_values("num_total", ascending=False)[:10]

,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12,num_total,value_total
11150,5d1c466a-9cfd-11e9-9897-000d3a38a36f,Android App,Desktop,2013-10-11,2021-04-30,2021-04-30,2020-12-24,200.0000,2.0000,684.9700,45220.1300,"[AKTIFCOCUK, ERKEK, COCUK, KADIN, AKTIFSPOR]",202.0000,45905.1000
7223,cba59206-9dd1-11e9-9897-000d3a38a36f,Android App,Android App,2013-02-21,2021-05-09,2021-05-09,2020-01-25,130.0000,1.0000,49.9900,12232.2500,"[AKTIFCOCUK, ERKEK, KADIN, AKTIFSPOR]",131.0000,12282.2400
8783,a57f4302-b1a8-11e9-89fa-000d3a38a36f,Android App,Offline,2019-08-07,2020-11-04,2020-09-07,2020-11-04,2.0000,109.0000,10239.4600,143.9800,"[AKTIFCOCUK, ERKEK, COCUK, KADIN, AKTIFSPOR]",111.0000,10383.4400
2619,fdbe8304-a7ab-11e9-a2fc-000d3a38a36f,Android App,Offline,2018-10-18,2020-06-30,2018-10-18,2020-06-30,1.0000,87.0000,8432.2500,139.9800,[ERKEK],88.0000,8572.2300
6322,329968c6-a0e2-11e9-a2fc-000d3a38a36f,Ios App,Ios App,2019-02-14,2021-04-05,2021-04-05,2020-02-17,2.0000,81.0000,3997.5500,242.8100,[ERKEK],83.0000,4240.3600
7613,73fd19aa-9e37-11e9-9897-000d3a38a36f,Ios App,Offline,2014-01-14,2021-05-18,2021-01-30,2021-05-18,81.0000,1.0000,1263.7600,32654.3400,"[ERKEK, COCUK, KADIN, AKTIFSPOR]",82.0000,33918.1000
9347,44d032ee-a0d4-11e9-a2fc-000d3a38a36f,Mobile,Mobile,2019-02-11,2021-02-11,2021-02-11,2020-12-24,3.0000,74.0000,4640.7700,543.2800,"[KADIN, AKTIFSPOR]",77.0000,5184.0500
10954,b27e241a-a901-11e9-a2fc-000d3a38a36f,Mobile,Mobile,2015-09-12,2021-04-01,2021-04-01,2019-10-08,72.0000,3.0000,292.9300,5004.9500,"[AKTIFCOCUK, ERKEK, KADIN, AKTIFSPOR]",75.0000,5297.8800
8068,d696c654-2633-11ea-8e1c-000d3a38a36f,Ios App,Ios App,2017-05-10,2021-04-13,2021-04-13,2019-08-15,69.0000,1.0000,82.4800,16836.0900,"[AKTIFCOCUK, ERKEK, COCUK, KADIN, AKTIFSPOR]",70.0000,16918.5700
7330,a4d534a2-5b1b-11eb-8dbd-000d3a38a36f,Desktop,Desktop,2020-02-16,2021-04-30,2021-04-30,2020-12-18,66.0000,4.0000,843.6800,17599.8900,"[ERKEK, KADIN, AKTIFSPOR]",70.0000,18443.5700


Adım 8: Veri ön hazırlık sürecini fonksiyonlaştırınız.

In [12]:
def data_prep(dataframe):
    dataframe["num_total"] = dataframe["order_num_total_ever_online"] + dataframe["order_num_total_ever_offline"]
    dataframe["value_total"] = dataframe["customer_value_total_ever_offline"] + dataframe["customer_value_total_ever_online"]
    date_columns = dataframe.columns[dataframe.columns.str.contains("date")]
    dataframe[date_columns] = dataframe[date_columns].apply(pd.to_datetime)
    return df

#Görev 2: RFM Metriklerinin Hesaplanması#

Adım 1: Recency, Frequency ve Monetary tanımlarını yapınız

In [ ]:
#Recency(yenilik : analizin yapıldığı tarih- müşterinein son satın aldığı tarih)
#Frequency(müşterinin toplan satın alması) 
#Monetary(parasal değer)

Adım 2: Müşteri özelinde Recency, Frequency ve Monetary metriklerini hesaplayınız.
-Recency değerini hesaplamak için analiz tarihini maksimum tarihten 2 gün sonrası seçebilirsiniz.
Adım 3: Hesapladığınız metrikleri rfm isimli bir değişkene atayınız.
Adım 4: Oluşturduğunuz metriklerin isimlerini recency, frequency ve monetary olarak değiştiriniz.

In [76]:
import datetime 
# işlemleri daha kolay yapmak için kütüphanemizi import ediyoruz.


In [81]:
analysis_date= df["last_order_date"].max() + datetime.timedelta(days=2) #2gün sonrası


In [83]:
analysis_date # yeni oluşturduğumuz analiz tarihini kontrol ediyoruz.


Timestamp('2021-06-01 00:00:00')

In [85]:
#-Recency değerini hesaplamak için analiz tarihini maksimum tarihten 2 gün sonrası seçebilirsiniz
rfm = pd.DataFrame()
rfm["customer_id"] = df["master_id"] # işlem yaparken colomnsisimlerini de değiştiriyoruz.
rfm["recency"] = (analysis_date - df["last_order_date"]).astype('timedelta64[D]') #astype('timedelta64[D]') ile gün cinsine çeviriyoruz.
rfm["frequency"] = df["num_total"]
rfm["monetary"] = df["value_total"]
rfm.head()

,customer_id,recency,frequency,monetary
0,cc294636-19f0-11eb-8d74-000d3a38a36f,95.00,5.00,939.37
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,105.00,21.00,2013.55
2,69b69676-1a40-11ea-941b-000d3a38a36f,186.00,5.00,585.32
3,1854e56c-491f-11eb-806e-000d3a38a36f,135.00,2.00,121.97
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,86.00,2.00,209.98


#Görev 3: RF Skorunun Hesaplanması

Adım 1: Recency, Frequency ve Monetary metriklerini qcut yardımı ile 1-5 arasında skorlara çeviriniz. 
Adım 2: Bu skorları recency_score, frequency_score ve monetary_score olarak kaydediniz.
Adım 3: recency_score ve frequency_score’u tek bir değişken olarak ifade ediniz ve RF_SCORE olarak kaydediniz. 

In [86]:
rfm["recency_score"] = pd.qcut(rfm['recency'], 5, labels=[5, 4, 3, 2, 1]) # recency'de küçük olana büyük puan verilir.

rfm["frequency_score"] = pd.qcut(rfm['frequency'].rank(method="first"), 5, labels=[1, 2, 3, 4, 5]) # küçük gördüğüne küçük puan ver.Frequency(müşterinin toplan satın alması) 
#.rank(method="first") kullanmamızın nedeni duplicate kayıtlar olabiliyor hata vermemesi için ilk bulduğu segmenti atıyoruz.
rfm["monetary_score"] = pd.qcut(rfm['monetary'], 5, labels=[1, 2, 3, 4, 5])#Monetary küçük olana küçük değeri ver(parasal değer)

rfm["RFM_SCORE"] = (rfm['recency_score'].astype(str) +
                    rfm['frequency_score'].astype(str))
rfm.head()

,customer_id,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RFM_SCORE
0,cc294636-19f0-11eb-8d74-000d3a38a36f,95.00,5.00,939.37,3,4,4,34
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,105.00,21.00,2013.55,3,5,5,35
2,69b69676-1a40-11ea-941b-000d3a38a36f,186.00,5.00,585.32,2,4,3,24
3,1854e56c-491f-11eb-806e-000d3a38a36f,135.00,2.00,121.97,3,1,1,31
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,86.00,2.00,209.98,3,1,1,31


#Görev 4: RF Skorunun Segment Olarak Tanımlanması

Adım 1: Oluşturulan RF skorları için segment tanımlamaları yapınız.
Adım 2: Aşağıdaki seg_map yardımı ile skorları segmentlere çeviriniz

Adım 2:

    r'[1-2][1-2]': 'hibernating', #1. elemanda 1 yada 2  görürsen , 2.elamanda 1 yada 2 gçrürsen hibernating' yaz demek
    r'[1-2][3-4]': 'at_Risk',
    r'[1-2]5': 'cant_loose',  # 2. elemanda  görürsen köşeli parantez - varsa yada demek.
    r'3[1-2]': 'about_to_sleep',
    r'33': 'need_attention',  # 1 elemanında ve 2. elemanında 3 görürsen bu isimldirmeyi yap
    r'[3-4][4-5]': 'loyal_customers',
    r'41': 'promising',
    r'51': 'new_customers',
    r'[4-5][2-3]': 'potential_loyalists',
    r'5[4-5]': 'champions'

In [87]:
seg_map = {
    r'[1-2][1-2]': 'hibernating', 
    r'[1-2][3-4]': 'at_Risk',
    r'[1-2]5': 'cant_loose',  
    r'3[1-2]': 'about_to_sleep',
    r'33': 'need_attention',  
    r'[3-4][4-5]': 'loyal_customers',
    r'41': 'promising',
    r'51': 'new_customers',
    r'[4-5][2-3]': 'potential_loyalists',
    r'5[4-5]': 'champions'
}

In [88]:
rfm['segment'] = rfm['RFM_SCORE'].replace(seg_map, regex=True)#RF skorları için segment tanımlamaları

In [90]:
rfm.head(10)#ilk 10 değere bakalım

,customer_id,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RFM_SCORE,segment
0,cc294636-19f0-11eb-8d74-000d3a38a36f,95.00,5.00,939.37,3,4,4,34,loyal_customers
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,105.00,21.00,2013.55,3,5,5,35,loyal_customers
2,69b69676-1a40-11ea-941b-000d3a38a36f,186.00,5.00,585.32,2,4,3,24,at_Risk
3,1854e56c-491f-11eb-806e-000d3a38a36f,135.00,2.00,121.97,3,1,1,31,about_to_sleep
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,86.00,2.00,209.98,3,1,1,31,about_to_sleep
5,e585280e-aae1-11e9-a2fc-000d3a38a36f,80.00,3.00,200.86,4,2,1,42,potential_loyalists
6,c445e4ee-6242-11ea-9d1a-000d3a38a36f,226.00,4.00,375.93,2,3,2,23,at_Risk
7,3f1b4dc8-8a7d-11ea-8ec0-000d3a38a36f,293.00,2.00,163.63,1,1,1,11,hibernating
8,cfbda69e-5b4f-11ea-aca7-000d3a38a36f,86.00,5.00,1054.69,3,4,5,34,loyal_customers
9,1143f032-440d-11ea-8b43-000d3a38a36f,240.00,2.00,165.96,1,1,1,11,hibernating


Görev 5: Aksiyon Zamanı


Adım 1: Segmentlerin recency, frequnecy ve monetary ortalamalarını inceleyiniz.

In [91]:
rfm[["segment", "recency", "frequency", "monetary"]].groupby("segment").agg(["mean"])

,recency,frequency,monetary
,mean,mean,mean
segment,,,
about_to_sleep,113.79,2.40,359.01
at_Risk,241.61,4.47,646.61
cant_loose,235.44,10.70,1474.47
champions,17.11,8.93,1406.63
hibernating,247.95,2.39,366.27
loyal_customers,82.59,8.37,1216.82
need_attention,113.83,3.73,562.14
new_customers,17.92,2.00,339.96


Adım 2: RFM analizi yardımıyla aşağıda verilen 2 case için ilgili profildeki müşterileri bulun ve müşteri id'lerini csv olarak kaydediniz.

a. FLO bünyesine yeni bir kadın ayakkabı markası dahil ediyor. Dahil ettiği markanın ürün fiyatları genel müşteri
tercihlerinin üstünde. Bu nedenle markanın tanıtımı ve ürün satışları için ilgilenecek profildeki müşterilerle özel olarak
iletişime geçmek isteniliyor. Sadık müşterilerinden(champions, loyal_customers) ve kadın kategorisinden alışveriş
yapan kişiler özel olarak iletişim kurulacak müşteriler. Bu müşterilerin id numaralarını csv dosyasına kaydediniz

In [100]:
target_customers = rfm[rfm["segment"].isin(["champions","loyal_customers"])]["customer_id"] # ıd numarasına göre champions","loyal_customers olan müşteri numaralarını getirdik

In [105]:
cust_ids = df[(df["master_id"].isin(target_customers)) &(df["interested_in_categories_12"].str.contains("KADIN"))]["master_id"]
#müşteri ıd sına göre yukarıdaki filtremizi ve interested_in_categories_12 colonunda içinde "kadın" yazanları müşteri ıdsına göre getir bunları birleştir dedik.

In [111]:
cust_ids.to_csv("yenii_marka_hedef_müşteri_id.csv", index=True) 
cust_ids.to_csv("yeni_marka_hedef_müşteri_id.csv", index=False)
# dosyamızın konumuna bu isimle csv olarak kaydettik.
#index=True ile index numaralı olarak kaydettik.
#index=False ile de index numarasız olarak kaydettik.



b. Erkek ve Çocuk ürünlerinde %40'a yakın indirim planlanmaktadır. Bu indirimle ilgili kategorilerle ilgilenen "geçmişte
iyi müşteri olan ama uzun süredir alışveriş yapmayan" "kaybedilmemesi gereken müşteriler", "uykuda olanlar" ve "yeni
gelen müşteriler" özel olarak hedef alınmak isteniyor. Uygun profildeki müşterilerin id'lerini csv dosyasına kaydediniz


In [112]:
target_customers_b = rfm[rfm["segment"].isin(["cant_loose","hibernating","new_customers,at_Risk"])]["customer_id"]
# tırnak içinde belirtilen müşterileri katogerisini id numarsına göre getirdik


In [118]:
cust_ids_b = df[(df["master_id"].isin(target_customers_b)) &(df["interested_in_categories_12"].str.contains("ERKEK"))|(df["interested_in_categories_12"].str.contains("COCUK"))]["master_id"]
#hedeflediğimiz müşterilere göre erkek ve cocuk katogerisine sahip olanları listeledik

In [120]:
cust_ids_b.to_csv("indirim_hedef_müşteri_id.csv", index=True) 
cust_ids_b.to_csv("indirimm_hedef_müşteri_id.csv", index=False)
# dosyamızın konumuna bu isimle csv olarak kaydettik.
#index=True ile index numaralı olarak kaydettik.
#index=False ile de index numarasız olarak kaydettik.